In [16]:
import json
import requests
import time
from datetime import datetime as dt
from datetime import timedelta
# import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ccxt
from tqdm import tqdm
from tvDatafeed import TvDatafeed, Interval

print(ccxt.__version__)

ftx = ccxt.ftx({'enableRateLimit':True})
binance = ccxt.binance({'enableRateLimit':True})
zm = ccxt.zipmex({'enableRateLimit':True})

API_HOST = 'https://api.bitkub.com'


1.64.7


In [17]:
def get_price_zm(sym):
    syms = sym +'/THB'
    return zm.fetch_ticker(syms)['close']


def get_thb():
    tv = TvDatafeed()
    symbols ='USDTHB'
    exchanges= 'GLOBALPRIME'
    timeframe = Interval.in_daily
    limit_bar =1
    df = tv.get_hist(symbol=symbols,exchange='OANDA',interval=timeframe,n_bars=limit_bar)
    return df['close'].values[0]

        
def bk_ticker(symbols='all'): 
    ''' symbols : List'''
    bit_kub_list = []
    all_res = {}
    req = '/api/market/ticker'
    url = API_HOST+req
    
    if symbols =='all':
        param ={'query':'all'}
        all_res = requests.get(url,params=param).json()

    else :
        for sym in symbols:
                syms= 'THB_'+ sym
                param ={'sym':syms}
    #             print(syms)
                try:
                    res = requests.get(url,params=param).json()
                    all_res.update(res)

                except Exception as e:
                    res= np.nan
                    all_res.update(res)
        return all_res

# OLD 
# def bk_ticker(symbols='all'): 
#     ''' symbols : List'''
#     bit_kub_list = []
#     all_res = {}

#     req = '/api/market/ticker'
#     url = API_HOST+req
#     if symbols =='all':
#         param ={'query':'all'}
#         all_res = requests.get(url,params=param).json()


#     else :
#         for i in symbols:
#             param ={'sym':i}
#             res = requests.get(url,params=param).json()
#             all_res.update(res)

In [18]:
bk_ticker(['CVC','BTC'])

{'THB_CVC': {'id': 14,
  'last': 10.49,
  'lowestAsk': 10.51,
  'highestBid': 10.49,
  'percentChange': 2.04,
  'baseVolume': 931225.82802177,
  'quoteVolume': 9612650.31,
  'isFrozen': 0,
  'high24hr': 10.55,
  'low24hr': 10.11,
  'change': 0.21,
  'prevClose': 10.49,
  'prevOpen': 10.28},
 'THB_BTC': {'id': 1,
  'last': 1476510.05,
  'lowestAsk': 1477999.99,
  'highestBid': 1476499,
  'percentChange': 3.17,
  'baseVolume': 409.09817317,
  'quoteVolume': 591523508.84,
  'isFrozen': 0,
  'high24hr': 1478000,
  'low24hr': 1416000,
  'change': 45361.05,
  'prevClose': 1476510.05,
  'prevOpen': 1431149}}

In [19]:
## CCXT Function 
def get_data(ex,symbols,timeframe='1d'):
    # 
    
      df= pd.DataFrame(ex.fetch_ohlcv(symbols,timeframe),columns=['date','open','high','low','close','volume'])
#       data.drop(columns='volume',inplace=True)
      df['date']=df['date'].apply(ex.iso8601)
      df['date']=pd.to_datetime(df['date'])
    #   df['date']=df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
    #   df['date']=pd.to_datetime(df['date'])
    #   df.dropna(inplace=True)

      df.set_index('date',inplace=True)
      return df




def order_book(exchange,symbol):
    try:
        res = exchange.fetchOrderBook(symbol)
        ex_name = exchange.name
        if ex_name == 'Zipmex' :
            # check size data bid ,ask and set to equal
                min_data = min(len(res['bids']), len(res['asks']))
                res['bids']  = res['bids'][:min_data]
                res['asks']  = res['bids'][:min_data]
        order_book = pd.DataFrame(res)
        b_size = [order_book['bids'][i][1] for i in range(len(order_book))]
        order_book['bids_size'] = b_size

        b_price =  [order_book['bids'][i][0] for i in range(len(order_book))]
        order_book['bids_price'] = b_price


        a_size = [order_book['asks'][i][1] for i in range(len(order_book))]
        a_price  = [order_book['asks'][i][0] for i in range(len(order_book))]
        order_book['asks_size'] = a_size
        order_book['asks_price'] = a_price


        order_book = order_book[['bids_price','bids_size','asks_price','asks_size']]
        order_book['value_bid'] = order_book['bids_price'] * order_book['bids_size']
        order_book['value_ask'] = order_book['asks_price'] * order_book['asks_size']
        bid = order_book['bids_price'][0]
        offer = order_book['asks_price'][0]
        res =exchange.fetch_ticker(symbol)
        last = res['last']
        asks=  res['ask']
        bids = res['bid']
        spread = (asks/bids) -1 # asks/bids -1 , (offer -bid) /offer
        spread_offer = last/offer -1
        spread_bid =bid/last -1

        ts=  dt.now()
    #     print(f' [{ts}] [{symbol}]  {bid} : {offer}  Spread @ {spread*100:.2f}%: Last{last}')
    #     print(f' [{ts}] [{symbol}]  Spread Bid{spread_bid*100:.2f} %   offer{spread_offer*100 :.2f} % ')
    except Exception as e:
        print(str(e))
        order_book = pd.DataFrame()
        order_book['bids_price'] = [np.nan]
        order_book['asks_price'] = [np.nan]
    return order_book


In [20]:
order_book(ftx,'XLM/USD')


ftx does not have market symbol XLM/USD


,bids_price,asks_price
0,NaN,NaN


In [21]:
def get_price_spread(exchange,name_list,end_symbol_str):

    ''' exchange =ccxt.ExchangeClass
        name_sym ['list'] = symbol name'''
    data = []
    ex_name = exchange.name 
    for name_sym in name_list:
        symbols= name_sym +'/'+end_symbol_str
        try:
            res= exchange.fetch_ticker(symbols)
            last = res['last']
            ask = res['ask']
            bid = res['bid']
            volume= res['quoteVolume']
            spread=  ((ask-bid) / ask)*100
        except Exception as e:
            res= []
            print(str(e))


    
#         if last >=1 :
        if res != []:
            dict_a = {'exchange':ex_name,'symbol':symbols,'name':name_sym,'last':last,'volume':volume,'bid':bid,'ask':ask,'spread':spread}
            data.append(dict_a)
        else :
            print('ERROR')
    return data
print(get_price_spread(zm,['DODO'],'USD'))


zipmex does not have market symbol DODO/USD
ERROR
[]


# Compare Price 2 Exchange

In [22]:
# Get data from zipmex exchange and compare usdthb realtime rate
sym_list = ['SOL','LUNA','ETH','FTT','BTC']
data = get_price_spread(zm,sym_list,'THB')
thb_usd = get_thb()

# convert symbol/thb to symbol/USD
for data_ in data :
    data_['last_usd'] = (data_['last'])/thb_usd 
    print(data_)

{'exchange': 'Zipmex', 'symbol': 'SOL/THB', 'name': 'SOL', 'last': 3410.59, 'volume': 8015188.0482, 'bid': 3409.86, 'ask': 3421.31, 'spread': 0.33466713042664414, 'last_usd': 101.76612758847048}
{'exchange': 'Zipmex', 'symbol': 'LUNA/THB', 'name': 'LUNA', 'last': 3140.38, 'volume': 12311755.1737, 'bid': 3140.0, 'ask': 3150.0, 'spread': 0.31746031746031744, 'last_usd': 93.70352688428716}
{'exchange': 'Zipmex', 'symbol': 'ETH/THB', 'name': 'ETH', 'last': 104560.58, 'volume': 36799338.222009, 'bid': 104549.47, 'ask': 104600.0, 'spread': 0.0483078393881442, 'last_usd': 3119.9075013427223}
{'exchange': 'Zipmex', 'symbol': 'FTT/THB', 'name': 'FTT', 'last': 1602.0, 'volume': 528762.8776, 'bid': 1606.46, 'ask': 1610.0, 'spread': 0.21987577639751327, 'last_usd': 47.80091901891746}
{'exchange': 'Zipmex', 'symbol': 'BTC/THB', 'name': 'BTC', 'last': 1471601.02, 'volume': 76799122.70898691, 'bid': 1471601.01, 'ask': 1472500.0, 'spread': 0.06105195246179902, 'last_usd': 43910.03819299397}


In [23]:
# Dict to DataFrame
df_zm = pd.DataFrame(data)
# delete thb bath columns
df_zm['last'] = df_zm['last_usd']
df_zm.drop(columns='last_usd',inplace=True)
df_zm

,exchange,symbol,name,last,volume,bid,ask,spread
0,Zipmex,SOL/THB,SOL,101.766128,8.015188e+06,3409.86,3421.31,0.334667
1,Zipmex,LUNA/THB,LUNA,93.703527,1.231176e+07,3140.00,3150.00,0.317460
2,Zipmex,ETH/THB,ETH,3119.907501,3.679934e+07,104549.47,104600.00,0.048308
3,Zipmex,FTT/THB,FTT,47.800919,5.287629e+05,1606.46,1610.00,0.219876
4,Zipmex,BTC/THB,BTC,43910.038193,7.679912e+07,1471601.01,1472500.00,0.061052


In [24]:
ftx_data = get_price_spread(ftx,df_zm.name.to_list(),'USD')
df_ftx= pd.DataFrame(ftx_data)
df_ftx

,exchange,symbol,name,last,volume,bid,ask,spread
0,FTX,SOL/USD,SOL,102.185,1.120751e+08,102.1675,102.185,0.017126
1,FTX,LUNA/USD,LUNA,94.070,2.368569e+07,94.0000,94.080,0.085034
2,FTX,ETH/USD,ETH,3128.500,5.174572e+08,3128.4000,3128.500,0.003196
3,FTX,FTT/USD,FTT,48.111,2.919327e+07,48.1100,48.111,0.002079
4,FTX,BTC/USD,BTC,44029.000,4.525881e+08,44028.0000,44029.000,0.002271


In [25]:
# Merge Dataframe
m_df  =pd.merge(df_zm, df_ftx, on=['name'],suffixes=['_zipmex','_ftx']).dropna()
# m_df['last_zipmex'] =  m_df['last_usd']
m_df = m_df[['name','last_zipmex','last_ftx']]
m_df['diff_pct']=(m_df['last_ftx'] - m_df['last_zipmex']) /m_df['last_zipmex']
m_df

,name,last_zipmex,last_ftx,diff_pct
0,SOL,101.766128,102.185,0.004116
1,LUNA,93.703527,94.070,0.003911
2,ETH,3119.907501,3128.500,0.002754
3,FTT,47.800919,48.111,0.006487
4,BTC,43910.038193,44029.000,0.002709


# Compare Fee

In [32]:
path = r'C:\Users\User\Desktop\fee.xlsx'
zm_fee = pd.read_excel(path, sheet_name='zipmex')
zm_fee['name'] =zm_fee[zm_fee.columns[0]].str.split().str[0]
zm_fee['fee'] =zm_fee[zm_fee.columns[1]].str.split().str[0]
zm_fee['exchange'] = 'Zipmex'
# Delete ['สินทรัพย์/โทเคน, ธรรมเนียม columns	']
zm_fee = zm_fee.iloc[:,2:]



In [33]:
# Get data from name list


zm_fee['symbol_name'] =zm_fee['name'] +'/THB'
price_list = []
for i in tqdm(zm_fee['symbol_name'].to_list()):
    last=  (zm.fetch_ticker(i)['last'])
    price_list.append(last)
zm_fee['last']  = price_list    

zm_fee['last'] = zm_fee['last'].round(3)
zm_fee['fee'] = zm_fee['fee'].astype('float')
zm_fee['value'] =zm_fee['last'] *zm_fee['fee'] 

# zm_fee.dropna().to_csv('zm_df.csv')

In [37]:
zm_fee.sort_values('value')

,name,fee,exchange,symbol_name,last,value
18,XLM,0.02000,Zipmex,XLM/THB,7.152,0.143040
19,SIX,1.00000,Zipmex,SIX/THB,4.650,4.650000
22,MATIC,0.10000,Zipmex,MATIC/THB,53.814,5.381400
3,XRP,0.50000,Zipmex,XRP/THB,28.144,14.072000
4,BCH,0.00500,Zipmex,BCH/THB,12067.000,60.335000
2,LTC,0.02500,Zipmex,LTC/THB,4166.104,104.152600
37,AFIN,100.00000,Zipmex,AFIN/THB,1.870,187.000000
42,YLDY,1300.00000,Zipmex,YLDY/THB,0.150,195.000000
21,TOK,150.00000,Zipmex,TOK/THB,1.419,212.850000
36,ALPHA,24.00000,Zipmex,ALPHA/THB,9.985,239.640000


In [29]:
# bitkub Fee rate 
fee_kub = pd.read_excel(path, sheet_name='bitkub')

fee_kub['name'] =fee_kub[fee_kub.columns[0]].str.split('(').str[1].str.split(')').str[0]
fee_kub['fee'] =fee_kub[fee_kub.columns[2]].str.split().str[0]
fee_kub['exchange'] ='bitkub'
fee_kub = fee_kub.iloc[:,4:][0:10]
# fee_kub['last'] =fee_kub['name'].apply(bk_ticker)
fee_kub

,name,fee,exchange
0,CVC,90.00000000,bitkub
1,BSV,0.0004,bitkub
2,ETH,0.00625000,bitkub
3,KUB,0.01,bitkub
4,COMP,0.21000000,bitkub
5,CRV,8.71000000,bitkub
6,AXS,0.56000000,bitkub
7,SAND,7.34000000,bitkub
8,XRP,0.25,bitkub
9,LINK,1.67000000,bitkub


In [ ]:
fee_kub['last'] =fee_kub['name'].to_list().apply(bk_ticker)

In [ ]:
ll = fee_kub['name'].to_list()
data = bk_ticker(ll)
price_datas
for i in data:
    price_data = data[i]['last']

In [ ]:
import math

# get data last from dic
fee_kub['last_n'] = 0
for i in range(len(fee_kub)):
    if (fee_kub['last'].iloc[i]) != {}:
#     if not math.isnan(fee_kub['last'].iloc[i]):
        lp = (fee_kub['last'].iloc[i]['last'])
        fee_kub['last_n'].iloc[i] = lp
#         print(i,lp)
    else:
        fee_kub['last_n'].iloc[i] = np.nan
        
fee_kub['last'] = fee_kub['last_n']
fee_kub.drop(columns='last_n',inplace=True)
fee_kub[['last','fee']] = fee_kub[['last','fee']].astype(float)
fee_kub['last']  = fee_kub['last'].round(3)
fee_kub['value'] = fee_kub['last'] * fee_kub['fee']


# fee_kub =fee_kub.sort_values('value',ascending=False).dropna()
fee_kub =fee_kub.dropna()

In [ ]:
fee_kub

In [15]:
m_df2 = pd.merge(zm_fee,fee_kub,on=['name'],suffixes=['_zm','_bk'])
m_df2

NameError: name 'zm_fee' is not defined

In [13]:
# quick view

def fast_view(name_sym:str,debug=True):
    ''' Compare Symbols on sym_list  multi exchange ['FTX , BINANCE , ZIPMEX ,BITKUB']''' 

    # Call price frome 
    p1=order_book(ftx,name_sym+'/USD')
    p2=order_book(binance,name_sym+'/USDT')
    p3= order_book(zm,name_sym+'/THB')
    
    bk_sym = [name_sym]

    p4 =  bk_ticker(bk_sym)


    #  get first bid from orderbook
    # display(p3.iloc[[0]],p2.iloc[[0]],p1.iloc[[0]])
    zm_usd  =float(round((p3.iloc[[0]]['asks_price'].values[0] / usd_thb),5)) #
    ftx_usd = p1.iloc[[0]]['asks_price'].values[0]
    bn_usd = p2.iloc[[0]]['asks_price'].values[0]
    if p4 != {}:
        bk_usd = float(round(p4['last'] / usd_thb,5))

    else:
        bk_usd = np.nan


    diff = (zm_usd/bn_usd)-1
    diff2 = (zm_usd/ftx_usd)-1
    diff3 = (ftx_usd/bn_usd)-1

    diff4 = (bk_usd/bn_usd)-1
    diff5 = (bk_usd/ftx_usd)-1
    diff6 = (bk_usd/zm_usd)-1
    if debug:
        print(symbol)
        print(f' Zipmex {zm_usd:.5f} Binance {bn_usd:.5f} {diff*100:.2f}%')
        print(f' Zipmex {zm_usd:.5f} FTX {ftx_usd:.5f}  {diff2*100:.2f}%')
        print(f' FTX {ftx_usd:.5f} Binance {bn_usd:.5f}  {diff3*100:.2f}%')
        print(f' BITKUB {bk_usd:.5f} Binance {bn_usd:.5f}  {diff4*100:.2f}%')
        print(f' BITKUB {bk_usd:.5f} FTX {ftx_usd:.5f}  {diff5*100:.2f}%')
        print(f' BITKUB {bk_usd:.5f} Zipmex {zm_usd:.5f}  {diff6*100:.2f}%')
    dict_data= {'symbol':name_sym,'zipmex':zm_usd,'binance':bn_usd,'ftx':ftx_usd,'bk_usd':bk_usd}
    return pd.DataFrame([dict_data])







In [14]:
m_df2 = m_df2.set_index('name').drop(['USDT','USDC'])
m_df2

NameError: name 'm_df2' is not defined

In [551]:

name_sym =['BNB','ETH','SOL','XRP','LTC','FTT']



all_df2 = pd.DataFrame()
for  i in m_df2.index.to_list():
    print(i)
    df=  fast_view(i,debug= False)
    all_df2  = all_df2.append(df)
all_df2

BTC
ETH
LTC
XRP
BCH
LINK
ENJ
OMG
YFI
MKR
COMP
SAND
XLM
ftx does not have market symbol XLM/USD
SIX
ftx does not have market symbol SIX/USD
binance does not have market symbol SIX/USDT
AXS
AAVE
SNX
BAT
FTT
UNI
CRV
ZRX
KNC
BAND
ALPHA
GALA
SUSHI


In [553]:
all_df2

,symbol,zipmex,binance,ftx,bk_usd
0,BTC,39644.23641,39160.27000,39165.000000,39728.80077
0,ETH,2683.51423,2650.64000,2650.700000,2684.64413
0,LTC,104.63861,103.40000,103.390000,104.55577
0,XRP,0.74405,0.73500,0.734900,0.74396
0,BCH,293.89772,290.40000,290.500000,294.69874
0,LINK,14.14537,14.03000,14.032000,14.19377
0,ENJ,1.42650,1.41100,1.411400,1.42686
0,OMG,4.00086,3.95800,3.958500,4.00295
0,YFI,20243.87624,20028.20000,20025.000000,20354.45116
0,MKR,1853.33783,1834.00000,1836.500000,1854.06338
